# TRADES FROM PYTHON

In [1]:
import numpy as np  # array
import os
import sys
import time as timer

Add `pytrades` folder

In [2]:
trades_path = os.path.abspath("/data1/borsato/code/local_trades/trades_dev_y/")
pytrades_path = os.path.join(trades_path, "pytrades")
sys.path.append(pytrades_path)
import pytrades
import constants as cst
import ancillary as anc

## integration parameters

In [3]:
t_start = 2454965.0 # start time of the integration
t_epoch = 2455088.212 # reference time of the orbital parameters
t_int   = 500.0 # total integration time from the start time

n_body = 3 # number of bodies (NOT PLANETS) in the system, that is star + planets
duration_check = 1 # to check or not the duration in this case it is a return flag, 1 means return it

set the integration arguments:  
- `n_body` **mandatory**  
- `duration_check` **mandatory**  
- `t_start` **optional**  
- `t_epoch` **optional**  
- `t_int` **optional**  

In [4]:
# without setting global value for t_start, t_epoch, t_int:
pytrades.args_init(
    n_body, 
    duration_check,
    t_epoch=None,
    t_start=None,
    t_int=None,
)
# or setting them
# pytrades.args_init(
#     n_body, 
#     duration_check,
#     t_epoch=t_epoch,
#     t_start=t_start,
#     t_int=t_int,
# )


## RV  
provides radial velocities info: time, rv (m/s), and error_rv (m/s)

In [5]:
t_rv = np.array([
    2455342.947836,
    2455344.061419,
    2455351.037415,
    2455352.011289,
    2455367.083543,
    2455376.930532,
])
rv_obs = np.array([
    17.15,
    2.23,
    -7.89,
    -4.61,
    -25.45,
    17.39,
])
erv_obs = np.array([
    2.51,
    2.74,
    2.36,
    2.21,
    2.49,
    2.61,
])
# rv_data = np.column_stack((t_rv, rv_obs, erv_obs))
n_rv = len(t_rv)

rv_setid = np.ones((n_rv)).astype(int)
n_rvset = len(np.unique(rv_setid))
print("Defined RVs with n_rv = ", n_rv)

Defined RVs with n_rv =  6


deallocation, this is a test, but the next fuction should deallocate RV data before re-allocate them

In [6]:
pytrades.deallocate_rv_dataset()

add the RV dataset to the common variable of TRADES

In [7]:
pytrades.set_rv_dataset(t_rv, rv_obs, erv_obs, rv_setid=rv_setid, n_rvset=n_rvset)

## TOs of planet b
define transit times (T0s) of planet b, that will be the `2nd` body, where the `1st` body is the star!

In [8]:
epo_b = np.array([
    -5,
    -4,
    -2,
    -1,
    0,
    2,
    3,
    4,
    5,
])

t0_b = np.array([
    2454977.24875,
    2454996.48240,
    2455034.95437,
    2455054.19058,
    2455073.43412,
    2455111.92589,
    2455131.17167,
    2455150.42951,
    2455169.68103,
])

et0_b = np.array([
    0.00059,
    0.00062,
    0.00052,
    0.00052,
    0.00072,
    0.00050,
    0.00048,
    0.00048,
    0.00046,
])

# t0b_data = np.column_stack((epo_b, t0_b, et0_b))
n_t0b = len(t0_b)

body_id = 2 # fortran numbering starting with 1 == star

add transits of planet b to the common variable of TRADES

In [9]:
pytrades.set_t0_dataset(body_id, epo_b, t0_b, et0_b)

## T0s planet c
define T0s for planet c, that is `3rd` body

In [10]:
epo_c = np.array([
    -3,
    -2,
    -1,
    0,
    1,
    2,
])

t0_c = np.array([
    2454969.30577,
    2455008.33086,
    2455047.33560,
    2455086.31251,
    2455125.26284,
    2455164.18168,
])
et0_c = np.array([
    0.00070,
    0.00061,
    0.00058,
    0.00059,
    0.00053,
    0.00055,
])
# t0c_data = np.column_stack((epo_c, t0_c, et0_c))
n_t0c = len(t0_c)

body_id = 3 # fortran numbering starting with 1 == star

add transits of planet c to the common variable of TRADES

In [11]:
pytrades.set_t0_dataset(body_id, epo_c, t0_c, et0_c)

if you try to repeat the `set_t0_dataset` it will deallocate and reallocate the data ... hopefully :)

In [12]:
pytrades.set_t0_dataset(body_id, epo_c, t0_c, et0_c) # yes it works...

## define the orbital parameter of system at the `t_epoch`

In [13]:
# parameter = array([star, planet b, planet c]) = array([body_1, body_2, body_3])
M_msun   = np.array([1.0, 0.136985 * cst.Mjups, 0.094348 * cst.Mjups]) # Masses in Solar unit
R_rsun   = np.array([1.1, 0.842 * cst.Rjups, 0.823 * cst.Rjups]) # Radii in Solar unit
P_day    = np.array([0.0, 19.238756, 38.986097]) # Periods in days
ecc      = np.array([0.0, 0.058, 0.068]) # eccentricities
argp_deg = np.array([0.0, 356.1, 167.6]) # argument of pericenters in degrees
mA_deg   = np.array([0.0, 3.8, 307.4]) # mean anonalies in degrees
inc_deg  = np.array([0.0, 88.55, 89.12]) # inclinations in degrees
lN_deg   = np.array([0.0, 0.0, 0.0])# longitude of ascending nodes in degrees

it is needed to define a flag vector to identify which bodies has to transit or not, the `star` does not, so first element has to be flagged to `False`.  
If you don't know if they transit or not set `star` to `False`, all `other bodies` to `True`.

In [14]:
transit_flag = np.array([False, True, True])

### OUTPUT:
- `rv_sim(n_rv)` in m/s  
- `body_T0_sim(n_T0s)` id of the body, from 2 to n_body, of each transit time    
- `epo_sim(n_T0s)` epoch number of the transit w.r.t. the linear ephemeris of the corresponding body  
- `t0_sim(n_T0s)` simulated transit time in days corresponding to epoch and body at the same row  
- `t14_sim(n_T0s)` simulated Total Duration as T4 - T1 in minutes  
- `kel_sim(n_T0s, 8)` keplerian elements for each transit time, each column an orbital elements:  
    - `kel_sim(, 0)` period in days  
    - `kel_sim(, 1)` semi-major axis in au  
    - `kel_sim(, 2)` eccentricity  
    - `kel_sim(, 3)` inclination in degrees  
    - `kel_sim(, 4)` mean anomaly in degrees  
    - `kel_sim(, 5)` argument of pericenter in degrees  
    - `kel_sim(, 6)` true anomaly in degrees  
    - `kel_sim(, 7)` longitude of ascending node in degrees  

let's run the orbital integration and get the simulated RVs and T0s (with orbital elements)

In [15]:
rv_sim, body_T0_sim, epo_sim, t0_sim, t14_sim, kel_sim = pytrades.kelements_to_rv_and_t0s(
    t_start,
    t_epoch,
    t_int,
    M_msun,
    R_rsun,
    P_day,
    ecc,
    argp_deg,
    mA_deg,
    inc_deg,
    lN_deg,
    transit_flag,
)

In [16]:
print("time RVobs RVsim_mps")
for to, rvo, rvs in zip(t_rv, rv_obs, rv_sim):
    print(to, rvo, rvs)

time RVobs RVsim_mps
2455342.947836 17.15 4.980736854926989
2455344.061419 2.23 1.8347252274533956
2455351.037415 -7.89 -1.6667847533897884
2455352.011289 -4.61 0.5108758858588707
2455367.083543 -25.45 -15.778435238604793
2455376.930532 17.39 11.142261075343548


In [17]:
print("body epoch T0o T0s T14_min")
for ibd, epo, t0o, t0s, t14 in zip(body_T0_sim, epo_sim, np.concatenate((t0_b, t0_c)), t0_sim, t14_sim):
    print(ibd, epo, t0o, t0s, t14)

body epoch T0o T0s T14_min
2 -5 2454977.24875 2454977.2458032337 265.6053379923105
2 -4 2454996.4824 2454996.4797574175 265.60894422233105
2 -2 2455034.95437 2455034.9506268245 265.6059803813696
2 -1 2455054.19058 2455054.1866168724 265.60070380568504
2 0 2455073.43412 2455073.431005566 265.5977024137974
2 2 2455111.92589 2455111.922470898 265.5827370285988
2 3 2455131.17167 2455131.1685427255 265.5742082744837
2 4 2455150.42951 2455150.4261464677 265.55913493037224
2 5 2455169.68103 2455169.6780878073 265.54903976619244
3 -3 2454969.30577 2454969.312543784 339.5433899760246
3 -2 2455008.33086 2455008.336975492 339.49311532080173
3 -1 2455047.3356 2455047.340681977 339.43354077637196
3 0 2455086.31251 2455086.3192708455 339.37173396348953
3 1 2455125.26284 2455125.2691809363 339.31634299457073
3 2 2455164.18168 2455164.1883075153 339.27657656371593


## create a log-likelyhood function such as:

In [ ]:
def fit_to_physical(fit_pars):
    
    # here convert the fit_pars into:
    # mass of all bodies (star, b, c, ...) in Msun,
    # radius Rsun,
    # period in day,
    # ecc,
    # argp in deg,
    # meana in deg,
    # inc deg,
    # longn deg,
    # use GLOBAL!
    
    return mass, radius, period, ecc, argp, meana, inc, longn 

def loglike_function(fit_pars):
    
    mass, radius, period, ecc, argp, meana, inc, longn = fit_to_physical(fit_pars)
    
    rv_sim, body_T0_sim, epo_sim, t0_sim, t14_sim, kel_sim = pytrades.kelements_to_rv_and_t0s(
        t_start, # GLOBAL
        t_epoch, # GLOBAL
        t_int, # GLOBAL
        mass, radius, period, ecc, argp, meana, inc, longn
        transit_flag, # GLOBAL
    )
    
    ## RV
    lnL_rv = 0
    # add trends, gp to rv_sim
    rv = rv_sim + trend_rv + gp_rv
    res_rv = rv_obs - rv # rv_obs GLOBAL
    # lnL_rv = f(res_rv, jitter, etc)
    
    ## T0s
    lnL_T0 = 0
    res_T0b = t0_b - t0_sim[body_T0_sim_sim==2] # t0_b GLOBAL
    res_T0c = t0_c - t0_sim[body_T0_sim_sim==3] # t0_c GLOBAL
    lnL_T0 = g(res_T0b, res_T0c)
    
    lnL = lnL_rv + lnL_T0
    
    return lnL

### try to deallocate a T0s dataset: planet c

In [ ]:
pytrades.deallocate_t0_dataset(3)

In [ ]:
rv_sim, body_T0_sim, epo_sim, t0_sim, t14_sim, kel_sim = pytrades.kelements_to_rv_and_t0s(
    t_start,
    t_epoch,
    t_int,
    M_msun,
    R_rsun,
    P_day,
    ecc,
    argp_deg,
    mA_deg,
    inc_deg,
    lN_deg,
    transit_flag,
)

In [ ]:
print("time RVobs RVsim_mps")
for to, rvo, rvs in zip(t_rv, rv_obs, rv_sim):
    print(to, rvo, rvs)

In [ ]:
print("body epoch T0 T14_min")
for ibd, epo, t0s, t14 in zip(body_T0_sim, epo_sim, t0_sim, t14_sim):
    print(ibd, epo, t0s, t14)

### testing only RVs output

In [ ]:
rv_sim = pytrades.kelements_to_rv(
    t_start,
    t_epoch,
    t_int,
    M_msun,
    R_rsun,
    P_day,
    ecc,
    argp_deg,
    mA_deg,
    inc_deg,
    lN_deg,
)

In [ ]:
print("time RVobs RVsim_mps")
for to, rvo, rvs in zip(t_rv, rv_obs, rv_sim):
    print(to, rvo, rvs)

### testing only T0s output  
re-adding T0s of planet c

In [ ]:
pytrades.set_t0_dataset(body_id, epo_c, t0_c, et0_c)

In [ ]:
body_T0_sim, epo_sim, t0_sim, t14_sim, kel_sim = pytrades.kelements_to_t0s(
    t_start,
    t_epoch,
    t_int,
    M_msun,
    R_rsun,
    P_day,
    ecc,
    argp_deg,
    mA_deg,
    inc_deg,
    lN_deg,
    transit_flag,
)

In [ ]:
print("body epoch T0 T14_min")
for ibd, epo, t0s, t14 in zip(body_T0_sim, epo_sim, t0_sim, t14_sim):
    print(ibd, epo, t0s, t14)